# Проект по SQL с подключением к базе данных

# В ходе проекта используются подазпросы и объединение таблиц для расчета средних значений переменных в данных

**Цель исследования:** изучить данные сервиса для чтения книг и выяснить, как его можно усовершенствовать

**Схема данных:**

![Google Drive Image](https://drive.google.com/uc?id=1KH5bAhBzIAjYmDaZOjtASePFirO6m3VI)


## Импортируем необходимый набор библиотек

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

## Подключимся к базе данных

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Изучим данные

In [3]:
# выгрузим датасет books
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# выгрузим датасет authors
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# выгрузим датасет ratings
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# выгрузим датасет reviews
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
# выгрузим датасет publishers
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


##### *Вывод:*
<p></p>

<div style="border:solid black 1px; padding: 20px">

Данные были выгружены и они соответсвуют схеме из описания данных
    
    
</div>

<p></p>

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [9]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,count
0,819


## Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [10]:
query = '''
SELECT b.book_id,
b.title,
COUNT(DISTINCT re.review_id) AS count_review,
AVG(ra.rating) AS average_rating
FROM books AS b
LEFT JOIN reviews AS re ON b.book_id = re.book_id
JOIN ratings AS ra ON b.book_id = ra.book_id
GROUP BY b.book_id
ORDER BY count_review DESC, average_rating DESC
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,book_id,title,count_review,average_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
query = '''
SELECT p.publisher,
COUNT(book_id) AS amount_of_books
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages> 50
GROUP BY p.publisher
ORDER BY amount_of_books DESC
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher,amount_of_books
0,Penguin Books,42


## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [12]:
query = '''
SELECT
      count.author AS author,
      AVG(count.avg_rating) AS avg_rating
FROM
(SELECT
     a.author AS author,
     b.book_id AS id,
     AVG(r.rating) AS avg_rating,
     COUNT(r.rating) AS cnt_rating
FROM authors AS a
INNER JOIN books AS b ON b.author_id = a.author_id
INNER JOIN ratings AS r ON r.book_id = b.book_id
GROUP BY author, id
HAVING COUNT(r.rating) >= 50) AS count
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query = '''
WITH users AS (SELECT COUNT (review_id) AS review_count
FROM reviews
WHERE username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT (rating_id) > 50)
GROUP BY username)
SELECT ROUND(AVG(review_count)) AS review_avg
FROM users
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,review_avg
0,24.0


## Общий вывод

<p></p>

<div style="border:solid black 1px; padding: 20px">

Исходя из исследования можно сделать следующие выводы:
    
- После 1 января 2000 года вышло 819 книг

- У книги Сумерки и двух книг о Гарри Потере больше всего обзоров
 
- Издательство Penguin books выпустило наибольшее число книг 42
    
- Автором с самой высокой средней оценкой книг является J.K. Rowling
    
- Средним количеством обзоров от пользователей, которые оставили больше 50 оценок, является 24 
    
</div>

<p></p>